In [1]:
import time

In [2]:
#Regular hash for images
from PIL import Image
import imagehash

def calculate_image_hash(image_path):
    """
    Calculate the hash value for the given image.

    Parameters:
        image_path (str): Path to the image file.

    Returns:
        str: Hash value representing the image.
    """
    # Open the image using PIL (Python Imaging Library)
    img = Image.open(image_path)

    # Compute the hash value using imagehash library
    hash_value = imagehash.average_hash(img)

    # Return the hexadecimal representation of the hash
    return str(hash_value)



In [3]:
#DEEP HASH code
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import hashlib

# Load the pre-trained VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False)

# Add custom top layers for hashing
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(128, activation='sigmoid')(x)  # Using sigmoid activation for hash code

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy')

# Function to preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values
    return img_array

# Function to generate the hash code for an image
def generate_image_hash(img_path):
    img_array = preprocess_image(img_path)
    hash_code = model.predict(img_array)
    # Convert hash code to hexadecimal representation
    hash_hex = hashlib.sha256(hash_code).hexdigest()
    return hash_hex



C:\Users\juanc\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\juanc\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\juanc\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\juanc\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# Example for deephashing
image_path = "Capture.jpg"
start_time = time.time()
image_hash = generate_image_hash(image_path)
print("Image Hash:", image_hash)
print("--- %s seconds ---" % (time.time() - start_time))

Image Hash: 5d9f3862ffffebe76886fc80e9cf388474b451c780360aa94cbc008e30ae997d
--- 0.44327878952026367 seconds ---


In [5]:
# Example for image hash
image_path = "Capture.jpg"
start_time = time.time()
hash_value = calculate_image_hash(image_path)
print("Hash value:", hash_value)
print("--- %s seconds ---" % (time.time() - start_time))

Hash value: ff7f7f001f1f0707
--- 0.0029926300048828125 seconds ---


In [6]:
#Generate private and public keys
from Crypto.PublicKey import RSA

def generate_key_pair():
    key = RSA.generate(2048)
    private_key = key.exportKey()
    public_key = key.publickey().exportKey()
    return private_key, public_key

ImportError: cannot import name 'RSA' from 'Crypto.PublicKey' (unknown location)

In [ ]:
##DIGITAL SIGNATURE 

import hashlib
from Crypto.Signature import PKCS1_v1_5
from Crypto.Hash import SHA256
from PIL import Image

def sign_image(image_path, private_key_path):
    # Load private key
    with open(private_key_path, 'rb') as f:
        private_key = RSA.importKey(f.read())

    # Read image file
    with open(image_path, 'rb') as f:
        image_data = f.read()

    # Compute hash of image data
    hash = SHA256.new(image_data)

    # Create signature
    signer = PKCS1_v1_5.new(private_key)
    signature = signer.sign(hash)

    return signature

def verify_signature(image_path, signature, public_key_path):
    # Load public key
    with open(public_key_path, 'rb') as f:
        public_key = RSA.importKey(f.read())

    # Read image file
    with open(image_path, 'rb') as f:
        image_data = f.read()

    # Compute hash of image data
    hash = SHA256.new(image_data)

    # Verify signature
    verifier = PKCS1_v1_5.new(public_key)
    if verifier.verify(hash, signature):
        print("Signature is valid.")
    else:
        print("Signature is not valid.")


In [ ]:
# Example usage
if __name__ == "__main__":

    # Sign image
    image_path = "Capture.JPG"
    
    start_time = time.time()
    signature = sign_image(image_path, "private_key.pem")

    # Verify signature
    verify_signature(image_path, signature, "public_key.pem")
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
##FAILED DEEP SIGNING ATTEMPT

def deep_hash(data):
    # Compute a deep hash using hashlib
    hash_obj = hashlib.sha512()
    hash_obj.update(data)
    return hash_obj.digest()

def sign_image(image_path, private_key_path):
    # Load private key
    with open(private_key_path, 'rb') as f:
        private_key = RSA.importKey(f.read())

    # Read image file
    with open(image_path, 'rb') as f:
        image_data = f.read()

    # Compute deep hash of image data
    deep_image_hash = deep_hash(image_data)

    # Create signature
    signer = PKCS1_v1_5.new(private_key)
    signature = signer.sign(deep_image_hash)

    return signature

def verify_signature(image_path, signature, public_key_path):
    # Load public key
    with open(public_key_path, 'rb') as f:
        public_key = RSA.importKey(f.read())

    # Read image file
    with open(image_path, 'rb') as f:
        image_data = f.read()

    # Compute deep hash of image data
    deep_image_hash = deep_hash(image_data)

    # Verify signature
    verifier = PKCS1_v1_5.new(public_key)
    if verifier.verify(deep_image_hash, signature):
        print("Signature is valid.")
    else:
        print("Signature is not valid.")

In [ ]:
# Example usage
if __name__ == "__main__":
    # Sign image
    image_path = "Capture.JPG"
    signature = sign_image(image_path, "private_key.pem")

    # Verify signature
    verify_signature(image_path, signature, "public_key.pem")

In [ ]:
##FAILED DEEP SIGNING ATTEMPT 2

import hashlib
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from PIL import Image
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize pixel values
    return img_array

def generate_image_hash(img_path):
    # Load the pre-trained VGG16 model without the top layers
    base_model = VGG16(weights='imagenet', include_top=False)

    # Add custom top layers for hashing
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(128, activation='sigmoid')(x)  # Using sigmoid activation for hash code

    # Define the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(), loss='binary_crossentropy')

    img_array = preprocess_image(img_path)
    hash_code = model.predict(img_array)
    # Convert hash code to hexadecimal representation
    hash_hex = hashlib.sha256(hash_code).hexdigest()
    return hash_hex

def sign_image(image_path, private_key_path):
    # Load private key
    with open(private_key_path, 'rb') as f:
        private_key = RSA.importKey(f.read())

    # Generate image hash
    image_hash = generate_image_hash(image_path)

    # Create signature
    signer = pkcs1_15.new(private_key)
    signature = signer.sign(hashlib.sha256(image_hash.encode()).digest())

    return signature

def verify_signature(image_path, signature, public_key_path):
    # Load public key
    with open(public_key_path, 'rb') as f:
        public_key = RSA.importKey(f.read())

    # Generate image hash
    image_hash = generate_image_hash(image_path)

    # Verify signature
    verifier = pkcs1_15.new(public_key)
    try:
        verifier.verify(hashlib.sha256(image_hash.encode()).digest(), signature)
        print("Signature is valid.")
    except (ValueError, TypeError):
        print("Signature is not valid.")



In [ ]:
# Example usage
if __name__ == "__main__":
    # Sign image
    image_path = "Capture.JPG"
    signature = sign_image(image_path, "private_key.pem")

    # Verify signature
    verify_signature(image_path, signature, "public_key.pem")